# Account Details
Iterate all existing user accounts and fill their details (updated). Additionally checks for suspensions on a daily basis.

In [ ]:
# required imports to access api_db, misc, misc.CONFIG, ...
import sys
sys.path = ['.', '..', '../..'] + sys.path
from collection import *

### Conditional Execution
Each file needs to verify if it should be executed or not based on the configurations (for some files this is not optional but all should have this section, even if it is tautological). Example:
```python
if not misc.CONFIG["collection"]["execute_this_script"]: exit()
```

In [ ]:
# Conditional execution
pass

<hr>
<h1 align="center">driver code</h1>

iterate every user, try to update their personal information -> upsert in db

In [ ]:
# Testing suspension detection and update in database
# try: 
#     api_db.api.GetUser(screen_name="fuckeveryword")
# except twitter.TwitterError as e:
#     print(handle_error_next_api(e, 1113868486168731648, "process_user:GetUser"))

In [ ]:
find_params = find_exclude_invalid({'screen_name': {'$exists': False}})
print("Finding with params:", find_params)

In [ ]:
total = api_db.col_users.count(find_params)
print("Total to process: %d" % total)

In [ ]:
def task(skip, limit):
    # find_params = find_exclude_invalid({'screen_name': {'$exists': False}}) # cannot reuse from above
    find_params = find_exclude_invalid({'screen_name': {'$exists': False}}) # without screen_name restriction checks for suspensions
    print('Finding with params:', find_params)
    users = api_db.col_users.find(find_params, {}, no_cursor_timeout=True).skip(skip).limit(limit)

    u = next_or_none(users)
    while u:
        print('details for: %s...' % u['_id'], end='', flush=True)
        try: 
            account = get_account_details(user_id=u['_id'])
            if account: upsert_user(user_to_db_format(account))
            else: print('[returned none]', end='', flush=True)
            print('Done')
            u = next_or_none(users)
        except twitter.TwitterError as e:
            if handle_error_next_api(e, u['_id'], 'process_user:GetUser'):
                print('Failed, retrying for user %s.' % u['_id'])    
            else: # failed but repeating will lead nowhere
                print('Failed, skipping.')

In [ ]:
dp = DynamicParallelism(total, task, "account_details", max_threads=misc.CONFIG["collection"]["max_threads"], batch_size=225_000)

In [ ]:
dp.run()

In [ ]:
# dp.remove_folder()

In [ ]:
# find_params = find_exclude_invalid({'screen_name': {'$exists': False}})
# print('Finding with params:', find_params)
# users = api_db.col_users.find(find_params, {}, no_cursor_timeout=True)#.limit(1)

# u = next_or_none(users)
# while u:
#     print('details for: %s...' % u['_id'], end='', flush=True)
#     try: 
#         account = get_account_details(user_id=u['_id'])
#         if account: upsert_user(user_to_db_format(account))
#         else: print('[returned none]', end='', flush=True)
#         print('Done')
#         u = next_or_none(users)
#     except twitter.TwitterError as e:
#         if handle_error_next_api(e, u['_id'], 'process_user:GetUser'):
#             print('Failed, retrying for user %s.' % u['_id'])    
#         else: # failed but repeating will lead nowhere
#             print('Failed, skipping.')

In [ ]:
print("DONE")